## Processing & Analytical goals:
1.Sessionize the web log by IP. Sessionize = aggregrate all page hits by visitor/IP during a session. https://en.wikipedia.org/wiki/Session_(web_analytics) <br>
2.Determine the average session time<br>
3.Determine unique URL visits per session. To clarify, count a hit to a unique URL only once per session.<br>
4.Find the most engaged users, ie the IPs with the longest session times

In [1]:
import os
os.getcwd()

'/Users/NK/Projects/pytm'

## Import Libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, countDistinct

In [6]:
# SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Processing & Analytical goals") \
    .config("spark.executor.pyspark.memory", "1gb") \
    .getOrCreate()
sc = spark.sparkContext

## Load Data

In [7]:
# read log file
rdd = sc.textFile('/Users/NK/Projects/pytm/2015_07_22_mktplace_shop_web_log_sample.log')
# split by " "
rdd = rdd.map(lambda line: line.split(" "))

 source : https://docs.aws.amazon.com/elasticloadbalancing/latest/classic/access-log-collection.html#access-log-entry-format <br>
columns = ["timestamp", "elb", "client:port", "backend:port", "request_pt", "backend_pt", "responce_pt",
 "elb_status_code","backend_status_code","received_bytes", "sent_bytes", "request", "user_agent",
"ssl_cipher","ssl_protocol"]

In [8]:
# Keep timestamp, ip and link 
# convert the RDD to DF 
df = rdd.map(lambda line: Row(timestamp=line[0], ipaddress=line[2].split(':')[0], link=line[12])).toDF()
df.show(5)

+---------------+--------------------+--------------------+
|      ipaddress|                link|           timestamp|
+---------------+--------------------+--------------------+
|123.242.248.130|https://paytm.com...|2015-07-22T09:00:...|
|  203.91.211.44|https://paytm.com...|2015-07-22T09:00:...|
|    1.39.32.179|https://paytm.com...|2015-07-22T09:00:...|
| 180.179.213.94|https://paytm.com...|2015-07-22T09:00:...|
| 120.59.192.208|https://paytm.com...|2015-07-22T09:00:...|
+---------------+--------------------+--------------------+
only showing top 5 rows



## Inspect data

In [6]:
print(df.dtypes)

[('ipaddress', 'string'), ('link', 'string'), ('timestamp', 'string')]


In [7]:
print((df.count(), len(df.columns)))

(1158500, 3)


In [8]:
# Number of distinct ipaddresses
df.agg(countDistinct(col("ipaddress")).alias("count")).show()

+-----+
|count|
+-----+
|90544|
+-----+



## 1. Sessionize the web log by IP 
    # time range = 15 min 
    # convert to datetime type
    # create 15 min time range
    # groupby time_range and ip
    # assign an Id to each session

In [9]:
df = df.withColumn('timestamp', df['timestamp'].cast(TimestampType()))
df_range = df.select(window("timestamp", "15 minutes").alias('time_range'),'timestamp',"ipaddress","link")
df_range.show(5, False)

+------------------------------------------+--------------------------+---------------+---------------------------------------------------------------------------------------------------+
|time_range                                |timestamp                 |ipaddress      |link                                                                                               |
+------------------------------------------+--------------------------+---------------+---------------------------------------------------------------------------------------------------+
|[2015-07-22 05:00:00, 2015-07-22 05:15:00]|2015-07-22 05:00:28.019143|123.242.248.130|https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null       |
|[2015-07-22 05:00:00, 2015-07-22 05:15:00]|2015-07-22 05:00:27.89458 |203.91.211.44  |https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2      |
|[2015-07-22 05:00:00, 2015-07-22 05:15:00]|2015-07-22 05:00

In [10]:
df_range_grpby = df_range.groupBy('time_range','ipaddress').count()
df_session= df_range_grpby.withColumn("session_Id", monotonically_increasing_id())
df_session.show(5,False)

+------------------------------------------+--------------+-----+----------+
|time_range                                |ipaddress     |count|session_Id|
+------------------------------------------+--------------+-----+----------+
|[2015-07-22 05:00:00, 2015-07-22 05:15:00]|1.38.17.231   |14   |0         |
|[2015-07-22 05:00:00, 2015-07-22 05:15:00]|161.51.16.10  |1    |1         |
|[2015-07-22 05:00:00, 2015-07-22 05:15:00]|117.213.93.103|3    |2         |
|[2015-07-22 05:00:00, 2015-07-22 05:15:00]|165.225.104.65|35   |3         |
|[2015-07-22 05:00:00, 2015-07-22 05:15:00]|1.39.46.218   |7    |4         |
+------------------------------------------+--------------+-----+----------+
only showing top 5 rows



In [11]:
df_sessions_timestamp = df_range.join(df_session,['time_range','ipaddress'])
df_sessions_timestamp.show(20)

+--------------------+---------------+--------------------+--------------------+-----+----------+
|          time_range|      ipaddress|           timestamp|                link|count|session_Id|
+--------------------+---------------+--------------------+--------------------+-----+----------+
|[2015-07-21 22:30...|  106.51.141.73|2015-07-21 22:44:...|https://paytm.com...|    1|       217|
|[2015-07-21 22:30...|107.167.109.115|2015-07-21 22:43:...|http://www.paytm....|    1|       211|
|[2015-07-21 22:30...|113.193.203.163|2015-07-21 22:41:...|https://paytm.com...|    1|       205|
|[2015-07-21 22:30...|  115.184.19.68|2015-07-21 22:43:...|https://paytm.com...|    1|       209|
|[2015-07-21 22:30...|  115.250.103.3|2015-07-21 22:44:...|https://paytm.com...|    3|       215|
|[2015-07-21 22:30...|  115.250.103.3|2015-07-21 22:44:...|https://paytm.com...|    3|       215|
|[2015-07-21 22:30...|  115.250.103.3|2015-07-21 22:44:...|https://paytm.com...|    3|       215|
|[2015-07-21 22:30..

## 2. Determine average session time
    # calculate first hit occurance
    # join with session dataframe
    # session_time = last hit - first fit
    # session_time = 0 if count = 1

In [12]:
first_hit = df_sessions_timestamp.groupBy("session_id").agg(min("timestamp").alias('first_hit'))
first_hit.show(5, False)

+----------+--------------------------+
|session_id|first_hit                 |
+----------+--------------------------+
|26        |2015-07-22 05:02:20.711268|
|29        |2015-07-22 05:02:32.186441|
|474       |2015-07-22 06:35:33.695588|
|8589934658|2015-07-22 05:04:23.673392|
|8589934965|2015-07-22 14:01:55.30041 |
+----------+--------------------------+
only showing top 5 rows



In [13]:
df_sessions_timestamp = first_hit.join(df_sessions_timestamp,['session_id'])

In [14]:
df_sessions_timestamp.dtypes

[('session_id', 'bigint'),
 ('first_hit', 'timestamp'),
 ('time_range', 'struct<start:timestamp,end:timestamp>'),
 ('ipaddress', 'string'),
 ('timestamp', 'timestamp'),
 ('link', 'string'),
 ('count', 'bigint')]

In [15]:
time_diff = (unix_timestamp(df_sessions_timestamp.timestamp)-unix_timestamp(df_sessions_timestamp.first_hit))
df_sessions_timestamp = df_sessions_timestamp.withColumn("time_diff", time_diff)
df_sessions_timestamp.dtypes

[('session_id', 'bigint'),
 ('first_hit', 'timestamp'),
 ('time_range', 'struct<start:timestamp,end:timestamp>'),
 ('ipaddress', 'string'),
 ('timestamp', 'timestamp'),
 ('link', 'string'),
 ('count', 'bigint'),
 ('time_diff', 'bigint')]

In [16]:
session_duration = df_sessions_timestamp.groupBy('session_id').agg(max("time_diff").alias("session_duration"))
session_duration.show(5)

+----------+----------------+
|session_id|session_duration|
+----------+----------------+
|        26|              13|
|        29|              33|
|       474|             226|
|8589934658|               0|
|8589934965|              10|
+----------+----------------+
only showing top 5 rows



In [19]:
df_sessions_timestamp = session_duration.join(df_sessions_timestamp,['session_id'])
df_sessions_timestamp.show(5)

+----------+----------------+--------------------+--------------------+------------+--------------------+--------------------+-----+---------+
|session_id|session_duration|           first_hit|          time_range|   ipaddress|           timestamp|                link|count|time_diff|
+----------+----------------+--------------------+--------------------+------------+--------------------+--------------------+-----+---------+
|        26|              13|2015-07-22 05:02:...|[2015-07-22 05:00...|218.248.82.9|2015-07-22 05:02:...|https://paytm.com...|    4|        0|
|        26|              13|2015-07-22 05:02:...|[2015-07-22 05:00...|218.248.82.9|2015-07-22 05:02:...|http://www.paytm....|    4|        3|
|        26|              13|2015-07-22 05:02:...|[2015-07-22 05:00...|218.248.82.9|2015-07-22 05:02:...|http://www.paytm....|    4|        8|
|        26|              13|2015-07-22 05:02:...|[2015-07-22 05:00...|218.248.82.9|2015-07-22 05:02:...|https://paytm.com...|    4|       13|

In [21]:
average_session_time = df_sessions_timestamp.groupBy().avg('session_duration')
average_session_time.show(5)

+---------------------+
|avg(session_duration)|
+---------------------+
|   141.58578161415625|
+---------------------+



## 3. Determine unique URL visits per session. To clarify, count a hit to a unique URL only once per session.

In [23]:
URL_per_session = df_sessions_timestamp.groupBy("session_id","link").count().distinct()
URL_per_session.show(20)

+----------+--------------------+-----+
|session_id|                link|count|
+----------+--------------------+-----+
|        26|https://paytm.com...|    2|
|        26|http://www.paytm....|    2|
|        29|https://paytm.com...|    1|
|        29|https://paytm.com...|    1|
|        29|https://paytm.com...|    1|
|        29|https://paytm.com...|    1|
|       474|https://paytm.com...|    2|
|       474|https://paytm.com...|    2|
|       474|https://paytm.com...|    2|
|       474|https://paytm.com...|    5|
|       474|https://paytm.com...|    3|
|       474|https://paytm.com...|    2|
|       474|https://paytm.com...|    1|
|       474|https://paytm.com...|    1|
|       474|https://paytm.com...|    1|
|       474|https://paytm.com...|    1|
|8589934658|https://paytm.com...|    1|
|8589934965|https://paytm.com...|    1|
|8589934965|https://paytm.com...|    1|
|8589935171|https://paytm.com...|    1|
+----------+--------------------+-----+
only showing top 20 rows



## 4. Find the most engaged users, ie the IPs with the longest session times

In [26]:
sort_user_as_duration = df_sessions_timestamp.sort(col("session_duration").desc())
engaged_users = sort_user_as_duration.distinct()
engaged_users.show(5)

+------------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+-----+---------+
|  session_id|session_duration|           first_hit|          time_range|     ipaddress|           timestamp|                link|count|time_diff|
+------------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+-----+---------+
|249108103236|             847|2015-07-22 06:30:...|[2015-07-22 06:30...|164.100.96.254|2015-07-22 06:44:...|https://paytm.com...|   95|      847|
|249108103236|             847|2015-07-22 06:30:...|[2015-07-22 06:30...|164.100.96.254|2015-07-22 06:30:...|https://paytm.com...|   95|        0|
|249108103236|             847|2015-07-22 06:30:...|[2015-07-22 06:30...|164.100.96.254|2015-07-22 06:30:...|https://paytm.com...|   95|        1|
|249108103236|             847|2015-07-22 06:30:...|[2015-07-22 06:30...|164.100.96.254|2015-07-22 06:31:...|https://p

In [27]:
engaged_users = engaged_users.select("ipaddress","session_id","session_duration")
engaged_users.show(5)

+--------------+------------+----------------+
|     ipaddress|  session_id|session_duration|
+--------------+------------+----------------+
|164.100.96.254|249108103236|             847|
|164.100.96.254|249108103236|             847|
|164.100.96.254|249108103236|             847|
|164.100.96.254|249108103236|             847|
|164.100.96.254|249108103236|             847|
+--------------+------------+----------------+
only showing top 5 rows

